In [1]:
import pandas as pd
import time
import altair as alt
import paretoset

#alt.renderers.enable('png')
#alt.renderers.set_embed_options(scale_factor=3.0)
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

# Reading Results

##### Cleaning Processed Results

In [2]:

def remove_rows_above_specific_value(df, column_name, value):
    return df[df[column_name] <= value]



# Plots - CaseB RES-Coupling + Energy Arbitrage

In [3]:


# Price and CO2eq bands
band_data_quartiles = pd.DataFrame({
    'Avg_Price': [0.033687, 0.079043],
    'Avg_CO2eq': [0.034770, 0.040760]
})
band_data_avg = pd.DataFrame({
    'Avg_Price': [0.065847],
    'Avg_CO2eq': [0.037910]
})

# Plot with error band
chart_band_price = alt.Chart(band_data_quartiles).mark_errorband(
    interpolate= 'linear',
    extent= 'ci',
    opacity= 0.25
).encode(
    y=alt.Y(
        'Avg_Price:Q',
        axis= alt.Axis(
            title=''
        )
    )
)
chart_rule_price_quartiles = alt.Chart(band_data_quartiles).mark_rule(
    opacity= 0.75,
    strokeDash= (5, 5)
).encode(
    y=alt.Y(
        'Avg_Price:Q',
        axis= alt.Axis(
            title=''
        )
    )
)
chart_rule_price_avg = alt.Chart(band_data_avg).mark_rule().encode(
    y=alt.Y(
        'Avg_Price:Q',
        axis= alt.Axis(
            title=''
        )
    )
)
chart_band_co2 = alt.Chart(band_data_quartiles).mark_errorband(
    interpolate= 'linear',
    extent= 'ci',
    opacity= 0.25
).encode(
    y=alt.Y(
        'Avg_CO2eq:Q',
        axis= alt.Axis(
            title=''
        )
    )
)
chart_rule_co2_quartiles = alt.Chart(band_data_quartiles).mark_rule(
    opacity= 0.75,
    strokeDash= (5, 5)
).encode(
    y=alt.Y(
        'Avg_CO2eq:Q',
        axis= alt.Axis(
            title=''
        )
    )
)
chart_rule_co2_avg = alt.Chart(band_data_avg).mark_rule().encode(
    y=alt.Y(
        'Avg_CO2eq:Q',
        axis= alt.Axis(
            title=''
        )
    )
)

#chart_band_price + chart_rule_price

### Raw Plots

In [7]:
shape_values = {'Economic Dispatch Optimisation': 'circle', 'Environmental Dispatch Optimisation': 'cross'}
color_mapping_earnings = {
    'positive': 'lightblue',  # Positive earnings
    'negative': 'red',    # Negative earnings
    'zero': 'gray'        # Zero earnings
}
color_mapping_abatement = {
    'positive': 'green',  # Positive earnings
    'negative': 'red',    # Negative earnings
    'zero': 'gray'        # Zero earnings
}
# color = alt.condition(
#     alt.datum.earnings_class == 'positive',  # Condition 1
#     alt.value(color_mapping_earnings['positive']),  # Color for positive
#     alt.condition(
#         alt.datum.earnings_class == 'negative', 
#         alt.value(color_mapping_earnings['negative']),  # Color for negative
#         alt.value(color_mapping_earnings['zero'])      # Default color (zero)
#     )
# )

#leer resultados
results_econ = pd.read_csv('../data/processed/CaseB/results_econ.csv', sep=';')
results_env = pd.read_csv('../data/processed/CaseB/results_env.csv', sep=';')
results = pd.read_csv('../data/processed/CaseB/results.csv', sep=';')


lines_lcoe_revenue_econ= alt.Chart(results_econ, title='LCOE - Revenue').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lcoe_revenue:Q', axis= alt.Axis(title='LCOE [EUR/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color('PV_Load_M:Q').scale(
        scheme='redblue',
        #domain= (0, 8)
    ),
)
lines_lcoe_revenue_env= alt.Chart(results_env, title='LCOE - Revenue').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lcoe_revenue:Q', axis= alt.Axis(title='LCOE [EUR/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            title= '',
            labelOpacity=0,
            gradientOpacity=0,
            symbolOpacity=0
        )
    ).scale(
        scheme='redblue',
        #domain= (0, 8)
    )
)
points_lcoe_revenue= alt.Chart(results).mark_point(
    size= 150,
    strokeWidth= 1,
    opacity= 0.25
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lcoe_revenue:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values())
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    # color= alt.Color(
    #     'earnings_class:N',
    #     legend= alt.Legend(
    #         title='Net Earnings',
    #         #title='',
    #         # labelOpacity= 0,
    #         # symbolOpacity= 0,
    #         # gradientOpacity= 0,
    #         # titleOpacity=0
    #     )
    # ).scale(
    #     scheme='redblue',
    #     #domain= (-1500000, 1500000)
    # ),
    color= alt.Color(
        'earnings_class:N',
        scale=alt.Scale(
            domain=list(color_mapping_earnings.keys()),
            range=list(color_mapping_earnings.values())
        ),
        legend= alt.Legend(
            title= 'Net Earnings'
        )
    ),
    size= alt.Size(
        'BESS_Load_M:Q',
        legend= alt.Legend(
            #title= '',
        )
    )
)


lines_lco2_abatement_econ= alt.Chart(results_econ, title='LCO2 - Abatement').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lco2_abatement:Q', axis= alt.Axis(title='LCO2 [kgCO2eq/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            title= 'PV-Load Multiple'
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (0, 8)
    ),
)
lines_lco2_abatement_env= alt.Chart(results_env, title='LCO2 - Abatement').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lco2_abatement:Q', axis= alt.Axis(title='LCO2 [kgCO2eq/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            title= '',
            labelOpacity=0,
            gradientOpacity=0,
            titleOpacity=0
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (0, 8)
    ),
)
points_lco2_abatement= alt.Chart(results).mark_point(
    size= 150,
    strokeWidth= 1,
    opacity= 0.15
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lco2_abatement:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values()),
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    # color= alt.Color(
    #     'co2abatement_class:N',
    #     legend= alt.Legend(
    #         title='CO2eq Abatement/Offset',
    #         # title='',
    #         # labelOpacity= 0,
    #         # symbolOpacity= 0,
    #         # gradientOpacity= 0
    #     )
    # ).scale(
    #     scheme='redyellowgreen',
    #     #domain= (-800000, 500000)
    # ),
    color= alt.Color(
        'co2abatement_class:N',
        scale=alt.Scale(
            domain=list(color_mapping_abatement.keys()),
            range=list(color_mapping_abatement.values())
        ),
        legend= alt.Legend(
            title= 'CO2eq Abatement/Offset'
        )
    ),
    size= alt.Size(
        'BESS_Load_M:Q'
    )
)


# alt.layer(lines_lcoe + points_lcoe_revenue)#.resolve_scale(shape= 'independent')


alt.hconcat(
    alt.layer(
        points_lcoe_revenue,
        lines_lcoe_revenue_econ,
        lines_lcoe_revenue_env
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    ),
    alt.layer(
        points_lco2_abatement,
        lines_lco2_abatement_econ,
        lines_lco2_abatement_env
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    )
).resolve_scale(
    #legend= 'independent',
    #scale= 'independent',
    shape= 'independent',
    color= 'independent'
)

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/ana

alt.HConcatChart(...)

In [8]:
shape_values = {'Economic Dispatch Optimisation': 'circle', 'Environmental Dispatch Optimisation': 'cross'}
color_mapping_earnings = {
    'positive': 'lightblue',  # Positive earnings
    'negative': 'red',    # Negative earnings
    'zero': 'gray'        # Zero earnings
}
color_mapping_abatement = {
    'positive': 'green',  # Positive earnings
    'negative': 'red',    # Negative earnings
    'zero': 'gray'        # Zero earnings
}


lines_lcoe_investor_econ= alt.Chart(results_econ, title='LCOE - Investor').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lcoe_investor:Q', axis= alt.Axis(title='LCOE [EUR/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color('PV_Load_M:Q').scale(
        scheme='redblue',
        #domain= (0, 8)
    ),
)
lines_lcoe_investor_env= alt.Chart(results_env, title='LCOE - Investor').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lcoe_investor:Q', axis= alt.Axis(title='LCOE [EUR/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            title= '',
            labelOpacity=0,
            gradientOpacity=0,
            symbolOpacity=0
        )
    ).scale(
        scheme='redblue',
        #domain= (0, 8)
    )
)
points_lcoe_investor= alt.Chart(results).mark_point(
    size= 150,
    strokeWidth= 1,
    opacity= 0.25
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lcoe_investor:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values())
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    # color= alt.Color(
    #     'earnings_class:N',
    #     legend= alt.Legend(
    #         title='Net Earnings',
    #         #title='',
    #         # labelOpacity= 0,
    #         # symbolOpacity= 0,
    #         # gradientOpacity= 0,
    #         # titleOpacity=0
    #     )
    # ).scale(
    #     scheme='redblue',
    #     #domain= (-1500000, 1500000)
    # ),
    color= alt.Color(
        'earnings_class:N',
        scale=alt.Scale(
            domain=list(color_mapping_earnings.keys()),
            range=list(color_mapping_earnings.values())
        ),
        legend= alt.Legend(
            title= 'Net Earnings'
        )
    ),
    size= alt.Size(
        'BESS_Load_M:Q',
        legend= alt.Legend(
            #title= '',
        )
    )
)


lines_lco2_investor_econ= alt.Chart(results_econ, title='LCO2 - Investor').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lco2_investor:Q', axis= alt.Axis(title='LCO2 [kgCO2eq/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            title= 'PV-Load Multiple'
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (0, 8)
    ),
)
lines_lco2_investor_env= alt.Chart(results_env, title='LCO2 - Investor').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lco2_investor:Q', axis= alt.Axis(title='LCO2 [kgCO2eq/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            title= '',
            labelOpacity=0,
            gradientOpacity=0,
            titleOpacity=0
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (0, 8)
    ),
)
points_lco2_investor= alt.Chart(results).mark_point(
    size= 150,
    strokeWidth= 1,
    opacity= 0.25
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lco2_investor:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values()),
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    # color= alt.Color(
    #     'co2abatement_class:N',
    #     legend= alt.Legend(
    #         title='CO2eq Abatement/Offset',
    #         # title='',
    #         # labelOpacity= 0,
    #         # symbolOpacity= 0,
    #         # gradientOpacity= 0
    #     )
    # ).scale(
    #     scheme='redyellowgreen',
    #     #domain= (-800000, 500000)
    # ),
    color= alt.Color(
        'co2abatement_class:N',
        scale=alt.Scale(
            domain=list(color_mapping_abatement.keys()),
            range=list(color_mapping_abatement.values())
        ),
        legend= alt.Legend(
            title= 'CO2eq Abatement/Offset'
        )
    ),
    size= alt.Size(
        'BESS_Load_M:Q'
    )
)


# alt.layer(lines_lcoe + points_lcoe)#.resolve_scale(shape= 'independent')


alt.hconcat(
    alt.layer(
        points_lcoe_investor,
        lines_lcoe_investor_econ,
        lines_lcoe_investor_env
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    ),
    alt.layer(
        points_lco2_investor,
        lines_lco2_investor_econ,
        lines_lco2_investor_env
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    )
).resolve_scale(
    #legend= 'independent',
    #scale= 'independent',
    shape= 'independent',
    color= 'independent'
)

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/ana

alt.HConcatChart(...)

In [9]:
shape_values = {'Economic Dispatch Optimisation': 'circle', 'Environmental Dispatch Optimisation': 'cross'}

lines_lcoe_enduser_econ= alt.Chart(results_econ, title='LCOE - End User').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lcoe_enduser:Q', axis= alt.Axis(title='LCOE [EUR/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color('PV_Load_M:Q').scale(
        scheme='redblue',
        #domain= (0, 8)
    ),
)
lines_lcoe_enduser_env= alt.Chart(results_env, title='LCOE - End User').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lcoe_enduser:Q', axis= alt.Axis(title='LCOE [EUR/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            title= '',
            labelOpacity=0,
            gradientOpacity=0,
            symbolOpacity=0
        )
    ).scale(
        scheme='redblue',
        #domain= (0, 8)
    )
)
points_lcoe_enduser= alt.Chart(results).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lcoe_enduser:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values())
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'savings:Q',
        legend= alt.Legend(
            title='Savings',
            #title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0,
            # titleOpacity=0
        )
    ).scale(
        scheme='blues',
        #domain= (-1500000, 1500000)
    ),
    size= alt.Size(
        'BESS_Load_M:Q',
        legend= alt.Legend(
            #title= '',
        )
    )
)


lines_lco2_enduser_econ= alt.Chart(results_econ, title='LCO2 - End User').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lco2_enduser:Q', axis= alt.Axis(title='LCO2 [kgCO2eq/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            title= 'PV-Load Multiple'
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (0, 8)
    ),
)
lines_lco2_enduser_env= alt.Chart(results_env, title='LCO2 - End User').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lco2_enduser:Q', axis= alt.Axis(title='LCO2 [kgCO2eq/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            title= '',
            labelOpacity=0,
            gradientOpacity=0,
            titleOpacity=0
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (0, 8)
    ),
)
points_lco2_enduser= alt.Chart(results).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lco2_enduser:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values()),
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'co2avoided:Q',
        legend= alt.Legend(
            title='CO2eq Avoided',
            # title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0
        )
    ).scale(
        scheme='yellowgreen',
        #domain= (-800000, 500000)
    ),
    size= alt.Size(
        'BESS_Load_M:Q'
    )
)


# alt.layer(lines_lcoe + points_lcoe)#.resolve_scale(shape= 'independent')


alt.hconcat(
    alt.layer(
        points_lcoe_enduser,
        lines_lcoe_enduser_econ,
        lines_lcoe_enduser_env,
        chart_band_price,
        chart_rule_price_quartiles,
        chart_rule_price_avg
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    ),
    alt.layer(
        points_lco2_enduser,
        lines_lco2_enduser_econ,
        lines_lco2_enduser_env,
        chart_band_co2,
        chart_rule_co2_quartiles,
        chart_rule_co2_avg
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    )
).resolve_scale(
    #legend= 'independent',
    #scale= 'independent',
    shape= 'independent',
    color= 'independent'
)

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/ana

alt.HConcatChart(...)

In [10]:
# Select only the columns of interest
columns_of_interest = ['PV_Load_M', 'BESS_Load_M', 'T_BESS', 'pv2load', 'pv2bess', 'pv2grid', 'bess2load', 'bess2grid', 'grid2load', 'grid2bess', 'pv2curtail']
subset_data = results_econ[columns_of_interest]

# Melt the DataFrame to long format
melted_data = subset_data.melt(id_vars=['PV_Load_M', 'BESS_Load_M', 'T_BESS'], var_name='source', value_name='value')

# Plot stacked bar chart
quant= alt.Chart(
    melted_data,
    title= alt.TitleParams(
        text= 'BESS-Load Multiples, binned',
        subtitle= 'Each bin represents a BESS-Load multiple, within which bars are offset for each PV-Load multiple according to the BESS duration capacity (left-to-right: 0 - 8 hours)',
        anchor= 'middle',
        fontSize= 30,
        subtitleFontSize= 20
    )
).mark_bar().encode(
    x=alt.X('PV_Load_M:O', title='PV-Load Multiples', axis=alt.Axis(offset= -5)),
    #x2=alt.X2('PV_Load_M:O', title='PV-Load Multiple'),# axis=alt.Axis(offset=-5)),
    #x=alt.X('T_BESS:N', title= 'BESS Duration', axis= alt.Axis(offset= -5)),
    xOffset= 'T_BESS:N',
    y=alt.Y('sum(value):Q', title='Quantity of Power flows', axis= alt.Axis(labels= False)),#.stack('normalize'),
    color=alt.Color('source:N', legend= alt.Legend(title= 'Power flows')),
    column= alt.Column('BESS_Load_M:N', title= '', header= alt.Header(labelFontSize= 20)),
    tooltip=['PV_Load_M', 'BESS_Load_M', 'T_BESS', 'source', 'value'],
    #size= alt.Size('T_BESS:Q').scale(domain= (-0.1, 2)),
    
).properties(
    width=300,
    height=600
).configure_axis(
    labelFontSize= 15,
    titleFontSize= 20,
).configure_legend(
    labelFontSize= 30,
    titleFontSize= 30,
    orient= 'bottom',
    padding= 5
)
# num= alt.Chart(melted_data).mark_bar().encode(
#     x=alt.X('PV_Load_M:O', title='PV-Load Multiple', axis=alt.Axis(offset=-5)),
#     y=alt.Y('sum(value):Q', title='Quantity'),
#     color='source:N',
#     column= alt.Column('BESS_Load_M:N', title= 'BESS-Load Multiple, binned'),
#     tooltip=['PV_Load_M', 'BESS_Load_M', 'source', 'value'],
#     size= alt.Size('T_BESS:N').scale(domain= (-0.1, 2))
# ).properties(
#     width=300,
#     height=600
# )

quant

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [11]:
# Select only the columns of interest
columns_of_interest = ['PV_Load_M', 'BESS_Load_M', 'T_BESS', 'pv2load', 'pv2bess', 'pv2grid', 'bess2load', 'bess2grid', 'grid2load', 'grid2bess', 'pv2curtail']
subset_data = results_econ[columns_of_interest]

# Melt the DataFrame to long format
melted_data = subset_data.melt(id_vars=['PV_Load_M', 'BESS_Load_M', 'T_BESS'], var_name='source', value_name='value')

# Plot stacked bar chart
quant= alt.Chart(
    melted_data,
    title= alt.TitleParams(
        text= 'BESS-Load Multiples, binned',
        subtitle= 'Each bin represents a BESS-Load multiple, within which bars are offset for each PV-Load multiple according to the BESS duration capacity (left-to-right: 0 - 8 hours)',
        anchor= 'middle',
        fontSize= 30,
        subtitleFontSize= 20
    )
).mark_bar().encode(
    x=alt.X('PV_Load_M:O', title='PV-Load Multiples', axis=alt.Axis(offset= -5)),
    #x2=alt.X2('PV_Load_M:O', title='PV-Load Multiple'),# axis=alt.Axis(offset=-5)),
    #x=alt.X('T_BESS:N', title= 'BESS Duration', axis= alt.Axis(offset= -5)),
    xOffset= 'T_BESS:N',
    y=alt.Y('sum(value):Q', title='Quantity of Power flows', axis= alt.Axis(labels= False)).stack('normalize'),
    color=alt.Color('source:N', legend= alt.Legend(title= 'Power flows')),
    column= alt.Column('BESS_Load_M:N', title= '', header= alt.Header(labelFontSize= 20)),
    tooltip=['PV_Load_M', 'BESS_Load_M', 'T_BESS', 'source', 'value'],
    #size= alt.Size('T_BESS:Q').scale(domain= (-0.1, 2)),
    
).properties(
    width=300,
    height=600
).configure_axis(
    labelFontSize= 15,
    titleFontSize= 20,
).configure_legend(
    labelFontSize= 30,
    titleFontSize= 30,
    orient= 'bottom',
    padding= 5
)
# num= alt.Chart(melted_data).mark_bar().encode(
#     x=alt.X('PV_Load_M:O', title='PV-Load Multiple', axis=alt.Axis(offset=-5)),
#     y=alt.Y('sum(value):Q', title='Quantity'),
#     color='source:N',
#     column= alt.Column('BESS_Load_M:N', title= 'BESS-Load Multiple, binned'),
#     tooltip=['PV_Load_M', 'BESS_Load_M', 'source', 'value'],
#     size= alt.Size('T_BESS:N').scale(domain= (-0.1, 2))
# ).properties(
#     width=300,
#     height=600
# )

quant

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [12]:
# Select only the columns of interest
columns_of_interest = ['PV_Load_M', 'BESS_Load_M', 'T_BESS', 'pv2load', 'pv2bess', 'pv2grid', 'bess2load', 'bess2grid', 'grid2load', 'grid2bess', 'pv2curtail']
subset_data = results_env[columns_of_interest]

# Melt the DataFrame to long format
melted_data = subset_data.melt(id_vars=['PV_Load_M', 'BESS_Load_M', 'T_BESS'], var_name='source', value_name='value')

# Plot stacked bar chart
quant= alt.Chart(
    melted_data,
    title= alt.TitleParams(
        text= 'BESS-Load Multiples, binned',
        subtitle= 'Each bin represents a BESS-Load multiple, within which bars are offset for each PV-Load multiple according to the BESS duration capacity (left-to-right: 0 - 8 hours)',
        anchor= 'middle',
        fontSize= 30,
        subtitleFontSize= 20
    )
).mark_bar().encode(
    x=alt.X('PV_Load_M:O', title='PV-Load Multiples', axis=alt.Axis(offset= -5)),
    #x2=alt.X2('PV_Load_M:O', title='PV-Load Multiple'),# axis=alt.Axis(offset=-5)),
    #x=alt.X('T_BESS:N', title= 'BESS Duration', axis= alt.Axis(offset= -5)),
    xOffset= 'T_BESS:N',
    y=alt.Y('sum(value):Q', title='Quantity of Power flows', axis= alt.Axis(labels= False)).stack('normalize'),
    color=alt.Color('source:N', legend= alt.Legend(title= 'Power flows')),
    column= alt.Column('BESS_Load_M:N', title= '', header= alt.Header(labelFontSize= 20)),
    tooltip=['PV_Load_M', 'BESS_Load_M', 'T_BESS', 'source', 'value'],
    #size= alt.Size('T_BESS:Q').scale(domain= (-0.1, 2)),
    
).properties(
    width=300,
    height=600
).configure_axis(
    labelFontSize= 15,
    titleFontSize= 20,
).configure_legend(
    labelFontSize= 30,
    titleFontSize= 30,
    orient= 'bottom',
    padding= 5
)
# num= alt.Chart(melted_data).mark_bar().encode(
#     x=alt.X('PV_Load_M:O', title='PV-Load Multiple', axis=alt.Axis(offset=-5)),
#     y=alt.Y('sum(value):Q', title='Quantity'),
#     color='source:N',
#     column= alt.Column('BESS_Load_M:N', title= 'BESS-Load Multiple, binned'),
#     tooltip=['PV_Load_M', 'BESS_Load_M', 'source', 'value'],
#     size= alt.Size('T_BESS:N').scale(domain= (-0.1, 2))
# ).properties(
#     width=300,
#     height=600
# )

quant

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [13]:
# Select only the columns of interest
columns_of_interest = ['PV_Load_M', 'BESS_Load_M', 'T_BESS', 'capex_pv', 'capex_bess', 'opex_pv', 'opex_bess', 'energy_procurement_costs_load', 'energy_procurement_costs_bess']
subset_data = results_econ[columns_of_interest]

# Melt the DataFrame to long format
melted_data = subset_data.melt(id_vars=['PV_Load_M', 'BESS_Load_M', 'T_BESS'], var_name='source', value_name='value')

# Plot stacked bar chart
quant= alt.Chart(
    melted_data,
    title= alt.TitleParams(
        text= 'BESS-Load Multiples, binned',
        subtitle= 'Each bin represents a BESS-Load multiple, within which bars are offset for each PV-Load multiple according to the BESS duration capacity (left-to-right: 0 - 8 hours)',
        anchor= 'middle',
        fontSize= 30,
        subtitleFontSize= 20
    )
).mark_bar().encode(
    x=alt.X('PV_Load_M:O', title='PV-Load Multiples', axis=alt.Axis(offset= -5)),
    #x2=alt.X2('PV_Load_M:O', title='PV-Load Multiple'),# axis=alt.Axis(offset=-5)),
    #x=alt.X('T_BESS:N', title= 'BESS Duration', axis= alt.Axis(offset= -5)),
    xOffset= 'T_BESS:N',
    y=alt.Y('sum(value):Q', title='Quantity Expenditures - normalised', axis= alt.Axis(labels= False)).stack('normalize'),
    color=alt.Color('source:N', legend= alt.Legend(title= 'Expenditures')),
    column= alt.Column('BESS_Load_M:N', title= '', header= alt.Header(labelFontSize= 20)),
    tooltip=['PV_Load_M', 'BESS_Load_M', 'T_BESS', 'source', 'value'],
    #size= alt.Size('T_BESS:Q').scale(domain= (-0.1, 2)),
    
).properties(
    width=300,
    height=600
).configure_axis(
    labelFontSize= 15,
    titleFontSize= 20,
).configure_legend(
    labelFontSize= 20,
    titleFontSize= 20,
    labelLimit= 400,
    orient= 'bottom',
    padding= 5
)
# num= alt.Chart(melted_data).mark_bar().encode(
#     x=alt.X('PV_Load_M:O', title='PV-Load Multiple', axis=alt.Axis(offset=-5)),
#     y=alt.Y('sum(value):Q', title='Quantity'),
#     color='source:N',
#     column= alt.Column('BESS_Load_M:N', title= 'BESS-Load Multiple, binned'),
#     tooltip=['PV_Load_M', 'BESS_Load_M', 'source', 'value'],
#     size= alt.Size('T_BESS:N').scale(domain= (-0.1, 2))
# ).properties(
#     width=300,
#     height=600
# )

quant

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [14]:
# Select only the columns of interest
columns_of_interest = ['PV_Load_M', 'BESS_Load_M', 'T_BESS', 'capex_pv', 'capex_bess', 'opex_pv', 'opex_bess', 'energy_procurement_costs_load', 'energy_procurement_costs_bess']
subset_data = results_env[columns_of_interest]

# Melt the DataFrame to long format
melted_data = subset_data.melt(id_vars=['PV_Load_M', 'BESS_Load_M', 'T_BESS'], var_name='source', value_name='value')

# Plot stacked bar chart
quant= alt.Chart(
    melted_data,
    title= alt.TitleParams(
        text= 'BESS-Load Multiples, binned',
        subtitle= 'Each bin represents a BESS-Load multiple, within which bars are offset for each PV-Load multiple according to the BESS duration capacity (left-to-right: 0 - 8 hours)',
        anchor= 'middle',
        fontSize= 30,
        subtitleFontSize= 20
    )
).mark_bar().encode(
    x=alt.X('PV_Load_M:O', title='PV-Load Multiples', axis=alt.Axis(offset= -5)),
    #x2=alt.X2('PV_Load_M:O', title='PV-Load Multiple'),# axis=alt.Axis(offset=-5)),
    #x=alt.X('T_BESS:N', title= 'BESS Duration', axis= alt.Axis(offset= -5)),
    xOffset= 'T_BESS:N',
    y=alt.Y('sum(value):Q', title='Quantity Expenditures - normalised', axis= alt.Axis(labels= False)).stack('normalize'),
    color=alt.Color('source:N', legend= alt.Legend(title= 'Expenditures')),
    column= alt.Column('BESS_Load_M:N', title= '', header= alt.Header(labelFontSize= 20)),
    tooltip=['PV_Load_M', 'BESS_Load_M', 'T_BESS', 'source', 'value'],
    #size= alt.Size('T_BESS:Q').scale(domain= (-0.1, 2)),
    
).properties(
    width=300,
    height=600
).configure_axis(
    labelFontSize= 15,
    titleFontSize= 20,
).configure_legend(
    labelFontSize= 20,
    titleFontSize= 20,
    labelLimit= 400,
    orient= 'bottom',
    padding= 5
)
# num= alt.Chart(melted_data).mark_bar().encode(
#     x=alt.X('PV_Load_M:O', title='PV-Load Multiple', axis=alt.Axis(offset=-5)),
#     y=alt.Y('sum(value):Q', title='Quantity'),
#     color='source:N',
#     column= alt.Column('BESS_Load_M:N', title= 'BESS-Load Multiple, binned'),
#     tooltip=['PV_Load_M', 'BESS_Load_M', 'source', 'value'],
#     size= alt.Size('T_BESS:N').scale(domain= (-0.1, 2))
# ).properties(
#     width=300,
#     height=600
# )

quant

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [15]:
# Select only the columns of interest
columns_of_interest = ['PV_Load_M', 'BESS_Load_M', 'T_BESS', 'capex_co2_pv', 'capex_co2_bess', 'opex_co2_pv', 'opex_co2_bess', 'co2burden_load', 'co2burden_bess']
subset_data = results_econ[columns_of_interest]

# Melt the DataFrame to long format
melted_data = subset_data.melt(id_vars=['PV_Load_M', 'BESS_Load_M', 'T_BESS'], var_name='source', value_name='value')

# Plot stacked bar chart
quant= alt.Chart(
    melted_data,
    title= alt.TitleParams(
        text= 'BESS-Load Multiples, binned',
        subtitle= 'Each bin represents a BESS-Load multiple, within which bars are offset for each PV-Load multiple according to the BESS duration capacity (left-to-right: 0 - 8 hours)',
        anchor= 'middle',
        fontSize= 30,
        subtitleFontSize= 20
    )
).mark_bar().encode(
    x=alt.X('PV_Load_M:O', title='PV-Load Multiples', axis=alt.Axis(offset= -5)),
    #x2=alt.X2('PV_Load_M:O', title='PV-Load Multiple'),# axis=alt.Axis(offset=-5)),
    #x=alt.X('T_BESS:N', title= 'BESS Duration', axis= alt.Axis(offset= -5)),
    xOffset= 'T_BESS:N',
    y=alt.Y('sum(value):Q', title='Quantity CO2eq Impact Sources - normalised', axis= alt.Axis(labels= False)).stack('normalize'),
    color=alt.Color('source:N', legend= alt.Legend(title= 'CO2eq Impact Sources')),
    column= alt.Column('BESS_Load_M:N', title= '', header= alt.Header(labelFontSize= 20)),
    tooltip=['PV_Load_M', 'BESS_Load_M', 'T_BESS', 'source', 'value'],
    #size= alt.Size('T_BESS:Q').scale(domain= (-0.1, 2)),
    
).properties(
    width=300,
    height=600
).configure_axis(
    labelFontSize= 15,
    titleFontSize= 20,
).configure_legend(
    labelFontSize= 20,
    titleFontSize= 20,
    labelLimit= 400,
    orient= 'bottom',
    padding= 5
)
# num= alt.Chart(melted_data).mark_bar().encode(
#     x=alt.X('PV_Load_M:O', title='PV-Load Multiple', axis=alt.Axis(offset=-5)),
#     y=alt.Y('sum(value):Q', title='Quantity'),
#     color='source:N',
#     column= alt.Column('BESS_Load_M:N', title= 'BESS-Load Multiple, binned'),
#     tooltip=['PV_Load_M', 'BESS_Load_M', 'source', 'value'],
#     size= alt.Size('T_BESS:N').scale(domain= (-0.1, 2))
# ).properties(
#     width=300,
#     height=600
# )

quant

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [16]:
# Select only the columns of interest
columns_of_interest = ['PV_Load_M', 'BESS_Load_M', 'T_BESS', 'capex_co2_pv', 'capex_co2_bess', 'opex_co2_pv', 'opex_co2_bess', 'co2burden_load', 'co2burden_bess']
subset_data = results_env[columns_of_interest]

# Melt the DataFrame to long format
melted_data = subset_data.melt(id_vars=['PV_Load_M', 'BESS_Load_M', 'T_BESS'], var_name='source', value_name='value')

# Plot stacked bar chart
quant= alt.Chart(
    melted_data,
    title= alt.TitleParams(
        text= 'BESS-Load Multiples, binned',
        subtitle= 'Each bin represents a BESS-Load multiple, within which bars are offset for each PV-Load multiple according to the BESS duration capacity (left-to-right: 0 - 8 hours)',
        anchor= 'middle',
        fontSize= 30,
        subtitleFontSize= 20
    )
).mark_bar().encode(
    x=alt.X('PV_Load_M:O', title='PV-Load Multiples', axis=alt.Axis(offset= -5)),
    #x2=alt.X2('PV_Load_M:O', title='PV-Load Multiple'),# axis=alt.Axis(offset=-5)),
    #x=alt.X('T_BESS:N', title= 'BESS Duration', axis= alt.Axis(offset= -5)),
    xOffset= 'T_BESS:N',
    y=alt.Y('sum(value):Q', title='Quantity CO2eq Impact Sources - normalised', axis= alt.Axis(labels= False)).stack('normalize'),
    color=alt.Color('source:N', legend= alt.Legend(title= 'Co2eq Impact Sources')),
    column= alt.Column('BESS_Load_M:N', title= '', header= alt.Header(labelFontSize= 20)),
    tooltip=['PV_Load_M', 'BESS_Load_M', 'T_BESS', 'source', 'value'],
    #size= alt.Size('T_BESS:Q').scale(domain= (-0.1, 2)),
    
).properties(
    width=300,
    height=600
).configure_axis(
    labelFontSize= 15,
    titleFontSize= 20,
).configure_legend(
    labelFontSize= 20,
    titleFontSize= 30,
    labelLimit= 400,
    orient= 'bottom',
    padding= 5
)
# num= alt.Chart(melted_data).mark_bar().encode(
#     x=alt.X('PV_Load_M:O', title='PV-Load Multiple', axis=alt.Axis(offset=-5)),
#     y=alt.Y('sum(value):Q', title='Quantity'),
#     color='source:N',
#     column= alt.Column('BESS_Load_M:N', title= 'BESS-Load Multiple, binned'),
#     tooltip=['PV_Load_M', 'BESS_Load_M', 'source', 'value'],
#     size= alt.Size('T_BESS:N').scale(domain= (-0.1, 2))
# ).properties(
#     width=300,
#     height=600
# )

quant

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

### Quick-Extraction Pareto Front

In [17]:
# Assuming you want to minimize both objectives
pareto_points_lcoe_revenue_1 = paretoset.paretoset(results[['lcoe_revenue', 'ress']].values, sense=["min", "min"])
pareto_points_lcoe_revenue_2 = paretoset.paretoset(results[['lcoe_revenue', 'ress']].values, sense=["min", "max"])
pareto_points_lco2_abatement_1 = paretoset.paretoset(results[['lco2_abatement', 'ress']].values, sense=["min", "min"])
pareto_points_lco2_abatement_2 = paretoset.paretoset(results[['lco2_abatement', 'ress']].values, sense=["min", "max"])

pareto_points_lcoe_investor_1 = paretoset.paretoset(results[['lcoe_investor', 'ress']].values, sense=["min", "min"])
pareto_points_lcoe_investor_2 = paretoset.paretoset(results[['lcoe_investor', 'ress']].values, sense=["min", "max"])
pareto_points_lco2_investor_1 = paretoset.paretoset(results[['lco2_investor', 'ress']].values, sense=["min", "min"])
pareto_points_lco2_investor_2 = paretoset.paretoset(results[['lco2_investor', 'ress']].values, sense=["min", "max"])

pareto_points_lcoe_enduser_1 = paretoset.paretoset(results[['lcoe_enduser', 'ress']].values, sense=["min", "min"])
pareto_points_lcoe_enduser_2 = paretoset.paretoset(results[['lcoe_enduser', 'ress']].values, sense=["min", "max"])
pareto_points_lco2_enduser_1 = paretoset.paretoset(results[['lco2_enduser', 'ress']].values, sense=["min", "min"])
pareto_points_lco2_enduser_2 = paretoset.paretoset(results[['lco2_enduser', 'ress']].values, sense=["min", "max"])

# Extract Pareto-optimal points
pareto_front_lcoe_revenue = pd.merge(results.iloc[pareto_points_lcoe_revenue_1], results.iloc[pareto_points_lcoe_revenue_2], how='outer')
pareto_front_lco2_abatement = pd.merge(results.iloc[pareto_points_lco2_abatement_1], results.iloc[pareto_points_lco2_abatement_2], how='outer')

pareto_front_lcoe_investor = pd.merge(results.iloc[pareto_points_lcoe_investor_1], results.iloc[pareto_points_lcoe_investor_2], how='outer')
pareto_front_lco2_investor = pd.merge(results.iloc[pareto_points_lco2_investor_1], results.iloc[pareto_points_lco2_investor_2], how='outer')

pareto_front_lcoe_enduser = pd.merge(results.iloc[pareto_points_lcoe_enduser_1], results.iloc[pareto_points_lcoe_enduser_2], how='outer')
pareto_front_lco2_enduser = pd.merge(results.iloc[pareto_points_lco2_enduser_1], results.iloc[pareto_points_lco2_enduser_2], how='outer')

In [18]:
pareto_lcoe_revenue= alt.Chart(pareto_front_lcoe_revenue).mark_line(
    color= 'black',
    size= 1.10
).encode(
    x= alt.X('ress:Q'),
    y= alt.Y('lcoe_revenue:Q')
)
pareto_lco2_abatement= alt.Chart(pareto_front_lco2_abatement).mark_line(
    color= 'black',
    size= 1.10
).encode(
    x= alt.X('ress:Q'),
    y= alt.Y('lco2_abatement:Q')
)

pareto_lcoe_investor= alt.Chart(pareto_front_lcoe_investor).mark_line(
    color= 'black',
    size= 1.10
).encode(
    x= alt.X('ress:Q'),
    y= alt.Y('lcoe_investor:Q')
)
pareto_lco2_investor= alt.Chart(pareto_front_lco2_investor).mark_line(
    color= 'black',
    size= 1.10
).encode(
    x= alt.X('ress:Q'),
    y= alt.Y('lco2_investor:Q')
)

pareto_lcoe_enduser= alt.Chart(pareto_front_lcoe_enduser).mark_line(
    color= 'black',
    size= 1.10
).encode(
    x= alt.X('ress:Q'),
    y= alt.Y('lcoe_enduser:Q')
)
pareto_lco2_enduser= alt.Chart(pareto_front_lco2_enduser).mark_line(
    color= 'black',
    size= 1.10
).encode(
    x= alt.X('ress:Q'),
    y= alt.Y('lco2_enduser:Q')
)

In [19]:
alt.hconcat(
    alt.layer(
        points_lcoe_revenue,
        lines_lcoe_revenue_econ,
        lines_lcoe_revenue_env,
        pareto_lcoe_revenue
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    ),
    alt.layer(
        points_lco2_abatement,
        lines_lco2_abatement_econ,
        lines_lco2_abatement_env,
        pareto_lco2_abatement
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    )
).resolve_scale(
    #legend= 'independent',
    #scale= 'independent',
    shape= 'independent',
    color= 'independent'
)

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/ana

alt.HConcatChart(...)

In [20]:
alt.hconcat(
    alt.layer(
        points_lcoe_investor,
        lines_lcoe_investor_econ,
        lines_lcoe_investor_env,
        pareto_lcoe_investor
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    ),
    alt.layer(
        points_lco2_investor,
        lines_lco2_investor_econ,
        lines_lco2_investor_env,
        pareto_lco2_investor
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    )
).resolve_scale(
    #legend= 'independent',
    #scale= 'independent',
    shape= 'independent',
    color= 'independent'
)

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/ana

alt.HConcatChart(...)

In [21]:
alt.hconcat(
    alt.layer(
        points_lcoe_enduser,
        lines_lcoe_enduser_econ,
        lines_lcoe_enduser_env,
        chart_band_price,
        chart_rule_price_quartiles,
        chart_rule_price_avg,
        pareto_lcoe_enduser
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    ),
    alt.layer(
        points_lco2_enduser,
        lines_lco2_enduser_econ,
        lines_lco2_enduser_env,
        chart_band_co2,
        chart_rule_co2_quartiles,
        chart_rule_co2_avg,
        pareto_lco2_enduser
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    )
).resolve_scale(
    #legend= 'independent',
    #scale= 'independent',
    shape= 'independent',
    color= 'independent'
)

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/ana

alt.HConcatChart(...)

In [22]:
## Plot only the points on the pareto fronts to see what multiples for PV, BESS, and BESS duration are on the front
left_up_revenue= alt.Chart(pareto_front_lcoe_revenue).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lcoe_revenue:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values())
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'BESS_Load_M:Q',
        legend= alt.Legend(
            #title='Savings',
            #title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0,
            # titleOpacity=0
        )
    ).scale(
        scheme='redblue',
        #domain= (-1500000, 1500000)
    ),
    size= alt.Size(
        'savings:Q',
        legend= alt.Legend(
            #title= '',
        )
    )
)
right_up_abatement= alt.Chart(pareto_front_lco2_abatement).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lco2_abatement:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values()),
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'BESS_Load_M:Q',
        legend= alt.Legend(
            title='CO2eq Avoided',
            # title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (-800000, 500000)
    ),
    size= alt.Size(
        'co2avoided:Q'
    )
)


left_down_revenue= alt.Chart(pareto_front_lcoe_revenue).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lcoe_revenue:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values())
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            #title='Savings',
            #title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0,
            # titleOpacity=0
        )
    ).scale(
        scheme='redblue',
        #domain= (-1500000, 1500000)
    ),
    size= alt.Size(
        'savings:Q',
        legend= alt.Legend(
            #title= '',
        )
    )
)
right_down_abatement= alt.Chart(pareto_front_lco2_abatement).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lco2_abatement:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values()),
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            #title='CO2eq Avoided',
            # title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (-800000, 500000)
    ),
    size= alt.Size(
        'co2avoided:Q'
    )
)


alt.vconcat(
    alt.hconcat(
        left_up_revenue,
        right_up_abatement
    ).resolve_scale(
        color= 'independent',
        shape= 'independent',
        size= 'independent'
    ),
    alt.hconcat(
        left_down_revenue,
        right_down_abatement
    ).resolve_scale(
        color= 'independent',
        shape= 'independent',
        size= 'independent'
    )
)

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/ana

alt.VConcatChart(...)

In [23]:
## Plot only the points on the pareto fronts to see what multiples for PV, BESS, and BESS duration are on the front
left_up_investor= alt.Chart(pareto_front_lcoe_investor).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lcoe_investor:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values())
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'BESS_Load_M:Q',
        legend= alt.Legend(
            #title='Savings',
            #title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0,
            # titleOpacity=0
        )
    ).scale(
        scheme='redblue',
        #domain= (-1500000, 1500000)
    ),
    size= alt.Size(
        'savings:Q',
        legend= alt.Legend(
            #title= '',
        )
    )
)
right_up_investor= alt.Chart(pareto_front_lco2_investor).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lco2_investor:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values()),
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'BESS_Load_M:Q',
        legend= alt.Legend(
            #title='CO2eq Avoided',
            # title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (-800000, 500000)
    ),
    size= alt.Size(
        'co2avoided:Q'
    )
)


left_down_investor= alt.Chart(pareto_front_lcoe_investor).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lcoe_investor:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values())
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            #title='Savings',
            #title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0,
            # titleOpacity=0
        )
    ).scale(
        scheme='redblue',
        #domain= (-1500000, 1500000)
    ),
    size= alt.Size(
        'savings:Q',
        legend= alt.Legend(
            #title= '',
        )
    )
)
right_down_investor= alt.Chart(pareto_front_lco2_investor).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lco2_investor:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values()),
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            #title='CO2eq Avoided',
            # title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (-800000, 500000)
    ),
    size= alt.Size(
        'co2avoided:Q'
    )
)


alt.vconcat(
    alt.hconcat(
        left_up_investor,
        right_up_investor
    ).resolve_scale(
        color= 'independent',
        shape= 'independent',
        size= 'independent'
    ),
    alt.hconcat(
        left_down_investor,
        right_down_investor
    ).resolve_scale(
        color= 'independent',
        shape= 'independent',
        size= 'independent'
    )
)

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/ana

alt.VConcatChart(...)

In [24]:
## Plot only the points on the pareto fronts to see what multiples for PV, BESS, and BESS duration are on the front
left_up_enduser= alt.Chart(pareto_front_lcoe_enduser).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lcoe_enduser:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values())
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'BESS_Load_M:Q',
        legend= alt.Legend(
            #title='Savings',
            #title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0,
            # titleOpacity=0
        )
    ).scale(
        scheme='redblue',
        #domain= (-1500000, 1500000)
    ),
    size= alt.Size(
        'savings:Q',
        legend= alt.Legend(
            #title= '',
        )
    )
)
right_up_enduser= alt.Chart(pareto_front_lco2_enduser).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lco2_enduser:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values()),
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'BESS_Load_M:Q',
        legend= alt.Legend(
            #title='CO2eq Avoided',
            # title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (-800000, 500000)
    ),
    size= alt.Size(
        'co2avoided:Q'
    )
)


left_down_enduser= alt.Chart(pareto_front_lcoe_enduser).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lcoe_enduser:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values())
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            #title='Savings',
            #title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0,
            # titleOpacity=0
        )
    ).scale(
        scheme='redblue',
        #domain= (-1500000, 1500000)
    ),
    size= alt.Size(
        'savings:Q',
        legend= alt.Legend(
            #title= '',
        )
    )
)
right_down_enduser= alt.Chart(pareto_front_lco2_enduser).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lco2_enduser:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values()),
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            #title='CO2eq Avoided',
            # title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (-800000, 500000)
    ),
    size= alt.Size(
        'co2avoided:Q'
    )
)


alt.vconcat(
    alt.hconcat(
        (left_up_enduser + chart_band_price + chart_rule_price_quartiles + chart_rule_price_avg),
        (right_up_enduser + chart_band_co2 + chart_rule_co2_quartiles + chart_rule_co2_avg)
    ).resolve_scale(
        color= 'independent',
        shape= 'independent',
        size= 'independent'
    ),
    alt.hconcat(
        (left_down_enduser + chart_band_price + chart_rule_price_quartiles + chart_rule_price_avg),
        (right_down_enduser + chart_band_co2 + chart_rule_co2_quartiles + chart_rule_co2_avg)
    ).resolve_scale(
        color= 'independent',
        shape= 'independent',
        size= 'independent'
    )
)

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/ana

alt.VConcatChart(...)

### Cut-off 120 EUR/MWh
(Since the largest point, that is still on the Paretto front, has an LCOE between 110 and 120 EUR/MWh)

In [25]:
results_econ_copy= results_econ.copy(deep= True)
results_env_copy= results_env.copy(deep= True)
results_copy= results.copy(deep= True)

results_econ= remove_rows_above_specific_value(results_econ, 'lcoe_investor', 0.12)
results_env= remove_rows_above_specific_value(results_env, 'lcoe_investor', 0.12)
results= remove_rows_above_specific_value(results, 'lcoe_investor', 0.12)

In [26]:
shape_values = {'Economic Dispatch Optimisation': 'circle', 'Environmental Dispatch Optimisation': 'cross'}
color_mapping_earnings = {
    'positive': 'lightblue',  # Positive earnings
    'negative': 'red',    # Negative earnings
    'zero': 'gray'        # Zero earnings
}
color_mapping_abatement = {
    'positive': 'green',  # Positive earnings
    'negative': 'red',    # Negative earnings
    'zero': 'gray'        # Zero earnings
}
# color = alt.condition(
#     alt.datum.earnings_class == 'positive',  # Condition 1
#     alt.value(color_mapping_earnings['positive']),  # Color for positive
#     alt.condition(
#         alt.datum.earnings_class == 'negative', 
#         alt.value(color_mapping_earnings['negative']),  # Color for negative
#         alt.value(color_mapping_earnings['zero'])      # Default color (zero)
#     )
# )





lines_lcoe_revenue_econ= alt.Chart(results_econ, title='LCOE - Revenue').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lcoe_revenue:Q', axis= alt.Axis(title='LCOE [EUR/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color('PV_Load_M:Q').scale(
        scheme='redblue',
        #domain= (0, 8)
    ),
)
lines_lcoe_revenue_env= alt.Chart(results_env, title='LCOE - Revenue').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lcoe_revenue:Q', axis= alt.Axis(title='LCOE [EUR/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            title= '',
            labelOpacity=0,
            gradientOpacity=0,
            symbolOpacity=0
        )
    ).scale(
        scheme='redblue',
        #domain= (0, 8)
    )
)
points_lcoe_revenue= alt.Chart(results).mark_point(
    size= 150,
    strokeWidth= 1,
    opacity= 0.25
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lcoe_revenue:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values())
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    # color= alt.Color(
    #     'earnings_class:N',
    #     legend= alt.Legend(
    #         title='Net Earnings',
    #         #title='',
    #         # labelOpacity= 0,
    #         # symbolOpacity= 0,
    #         # gradientOpacity= 0,
    #         # titleOpacity=0
    #     )
    # ).scale(
    #     scheme='redblue',
    #     #domain= (-1500000, 1500000)
    # ),
    color= alt.Color(
        'earnings_class:N',
        scale=alt.Scale(
            domain=list(color_mapping_earnings.keys()),
            range=list(color_mapping_earnings.values())
        ),
        legend= alt.Legend(
            title= 'Net Earnings'
        )
    ),
    size= alt.Size(
        'BESS_Load_M:Q',
        legend= alt.Legend(
            #title= '',
        )
    )
)


lines_lco2_abatement_econ= alt.Chart(results_econ, title='LCO2 - Abatement').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lco2_abatement:Q', axis= alt.Axis(title='LCO2 [kgCO2eq/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            title= 'PV-Load Multiple'
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (0, 8)
    ),
)
lines_lco2_abatement_env= alt.Chart(results_env, title='LCO2 - Abatement').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lco2_abatement:Q', axis= alt.Axis(title='LCO2 [kgCO2eq/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            title= '',
            labelOpacity=0,
            gradientOpacity=0,
            titleOpacity=0
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (0, 8)
    ),
)
points_lco2_abatement= alt.Chart(results).mark_point(
    size= 150,
    strokeWidth= 1,
    opacity= 0.15
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lco2_abatement:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values()),
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    # color= alt.Color(
    #     'co2abatement_class:N',
    #     legend= alt.Legend(
    #         title='CO2eq Abatement/Offset',
    #         # title='',
    #         # labelOpacity= 0,
    #         # symbolOpacity= 0,
    #         # gradientOpacity= 0
    #     )
    # ).scale(
    #     scheme='redyellowgreen',
    #     #domain= (-800000, 500000)
    # ),
    color= alt.Color(
        'co2abatement_class:N',
        scale=alt.Scale(
            domain=list(color_mapping_abatement.keys()),
            range=list(color_mapping_abatement.values())
        ),
        legend= alt.Legend(
            title= 'CO2eq Abatement/Offset'
        )
    ),
    size= alt.Size(
        'BESS_Load_M:Q'
    )
)


# alt.layer(lines_lcoe + points_lcoe_revenue)#.resolve_scale(shape= 'independent')


alt.hconcat(
    alt.layer(
        points_lcoe_revenue,
        lines_lcoe_revenue_econ,
        lines_lcoe_revenue_env
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    ),
    alt.layer(
        points_lco2_abatement,
        lines_lco2_abatement_econ,
        lines_lco2_abatement_env
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    )
).resolve_scale(
    #legend= 'independent',
    #scale= 'independent',
    shape= 'independent',
    color= 'independent'
)

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/ana

alt.HConcatChart(...)

In [27]:
shape_values = {'Economic Dispatch Optimisation': 'circle', 'Environmental Dispatch Optimisation': 'cross'}
color_mapping_earnings = {
    'positive': 'lightblue',  # Positive earnings
    'negative': 'red',    # Negative earnings
    'zero': 'gray'        # Zero earnings
}
color_mapping_abatement = {
    'positive': 'green',  # Positive earnings
    'negative': 'red',    # Negative earnings
    'zero': 'gray'        # Zero earnings
}


lines_lcoe_investor_econ= alt.Chart(results_econ, title='LCOE - Investor').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lcoe_investor:Q', axis= alt.Axis(title='LCOE [EUR/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color('PV_Load_M:Q').scale(
        scheme='redblue',
        #domain= (0, 8)
    ),
)
lines_lcoe_investor_env= alt.Chart(results_env, title='LCOE - Investor').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lcoe_investor:Q', axis= alt.Axis(title='LCOE [EUR/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            title= '',
            labelOpacity=0,
            gradientOpacity=0,
            symbolOpacity=0
        )
    ).scale(
        scheme='redblue',
        #domain= (0, 8)
    )
)
points_lcoe_investor= alt.Chart(results).mark_point(
    size= 150,
    strokeWidth= 1,
    opacity= 0.25
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lcoe_investor:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values())
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    # color= alt.Color(
    #     'earnings_class:N',
    #     legend= alt.Legend(
    #         title='Net Earnings',
    #         #title='',
    #         # labelOpacity= 0,
    #         # symbolOpacity= 0,
    #         # gradientOpacity= 0,
    #         # titleOpacity=0
    #     )
    # ).scale(
    #     scheme='redblue',
    #     #domain= (-1500000, 1500000)
    # ),
    color= alt.Color(
        'earnings_class:N',
        scale=alt.Scale(
            domain=list(color_mapping_earnings.keys()),
            range=list(color_mapping_earnings.values())
        ),
        legend= alt.Legend(
            title= 'Net Earnings'
        )
    ),
    size= alt.Size(
        'BESS_Load_M:Q',
        legend= alt.Legend(
            #title= '',
        )
    )
)


lines_lco2_investor_econ= alt.Chart(results_econ, title='LCO2 - Investor').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lco2_investor:Q', axis= alt.Axis(title='LCO2 [kgCO2eq/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            title= 'PV-Load Multiple'
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (0, 8)
    ),
)
lines_lco2_investor_env= alt.Chart(results_env, title='LCO2 - Investor').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lco2_investor:Q', axis= alt.Axis(title='LCO2 [kgCO2eq/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            title= '',
            labelOpacity=0,
            gradientOpacity=0,
            titleOpacity=0
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (0, 8)
    ),
)
points_lco2_investor= alt.Chart(results).mark_point(
    size= 150,
    strokeWidth= 1,
    opacity= 0.25
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lco2_investor:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values()),
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    # color= alt.Color(
    #     'co2abatement_class:N',
    #     legend= alt.Legend(
    #         title='CO2eq Abatement/Offset',
    #         # title='',
    #         # labelOpacity= 0,
    #         # symbolOpacity= 0,
    #         # gradientOpacity= 0
    #     )
    # ).scale(
    #     scheme='redyellowgreen',
    #     #domain= (-800000, 500000)
    # ),
    color= alt.Color(
        'co2abatement_class:N',
        scale=alt.Scale(
            domain=list(color_mapping_abatement.keys()),
            range=list(color_mapping_abatement.values())
        ),
        legend= alt.Legend(
            title= 'CO2eq Abatement/Offset'
        )
    ),
    size= alt.Size(
        'BESS_Load_M:Q'
    )
)


# alt.layer(lines_lcoe + points_lcoe)#.resolve_scale(shape= 'independent')


alt.hconcat(
    alt.layer(
        points_lcoe_investor,
        lines_lcoe_investor_econ,
        lines_lcoe_investor_env
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    ),
    alt.layer(
        points_lco2_investor,
        lines_lco2_investor_econ,
        lines_lco2_investor_env
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    )
).resolve_scale(
    #legend= 'independent',
    #scale= 'independent',
    shape= 'independent',
    color= 'independent'
)

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/ana

alt.HConcatChart(...)

In [28]:
shape_values = {'Economic Dispatch Optimisation': 'circle', 'Environmental Dispatch Optimisation': 'cross'}

lines_lcoe_enduser_econ= alt.Chart(results_econ, title='LCOE - End User').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lcoe_enduser:Q', axis= alt.Axis(title='LCOE [EUR/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color('PV_Load_M:Q').scale(
        scheme='redblue',
        #domain= (0, 8)
    ),
)
lines_lcoe_enduser_env= alt.Chart(results_env, title='LCOE - End User').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lcoe_enduser:Q', axis= alt.Axis(title='LCOE [EUR/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            title= '',
            labelOpacity=0,
            gradientOpacity=0,
            symbolOpacity=0
        )
    ).scale(
        scheme='redblue',
        #domain= (0, 8)
    )
)
points_lcoe_enduser= alt.Chart(results).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lcoe_enduser:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values())
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'savings:Q',
        legend= alt.Legend(
            title='Savings',
            #title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0,
            # titleOpacity=0
        )
    ).scale(
        scheme='blues',
        #domain= (-1500000, 1500000)
    ),
    size= alt.Size(
        'BESS_Load_M:Q',
        legend= alt.Legend(
            #title= '',
        )
    )
)


lines_lco2_enduser_econ= alt.Chart(results_econ, title='LCO2 - End User').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lco2_enduser:Q', axis= alt.Axis(title='LCO2 [kgCO2eq/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            title= 'PV-Load Multiple'
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (0, 8)
    ),
)
lines_lco2_enduser_env= alt.Chart(results_env, title='LCO2 - End User').mark_line(
    size= 1.5
).encode(
    x= alt.X('ress:Q', axis= alt.Axis(title='Renewable Energy Self Sufficiency [%]')),
    y= alt.Y('lco2_enduser:Q', axis= alt.Axis(title='LCO2 [kgCO2eq/kWh]')),
    # shape= alt.Shape(
    #     'earnings_class:N',
    #     scale=alt.Scale(
    #         domain=list(shape_values.keys()),
    #         range=list(shape_values.values())
    #     )
    # ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            title= '',
            labelOpacity=0,
            gradientOpacity=0,
            titleOpacity=0
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (0, 8)
    ),
)
points_lco2_enduser= alt.Chart(results).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lco2_enduser:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values()),
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'co2avoided:Q',
        legend= alt.Legend(
            title='CO2eq Avoided',
            # title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0
        )
    ).scale(
        scheme='yellowgreen',
        #domain= (-800000, 500000)
    ),
    size= alt.Size(
        'BESS_Load_M:Q'
    )
)


# alt.layer(lines_lcoe + points_lcoe)#.resolve_scale(shape= 'independent')


alt.hconcat(
    alt.layer(
        points_lcoe_enduser,
        lines_lcoe_enduser_econ,
        lines_lcoe_enduser_env,
        chart_band_price,
        chart_rule_price_quartiles,
        chart_rule_price_avg
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    ),
    alt.layer(
        points_lco2_enduser,
        lines_lco2_enduser_econ,
        lines_lco2_enduser_env,
        chart_band_co2,
        chart_rule_co2_quartiles,
        chart_rule_co2_avg
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    )
).resolve_scale(
    #legend= 'independent',
    #scale= 'independent',
    shape= 'independent',
    color= 'independent'
)

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/ana

alt.HConcatChart(...)

In [29]:
# Select only the columns of interest
columns_of_interest = ['PV_Load_M', 'BESS_Load_M', 'T_BESS', 'pv2load', 'pv2bess', 'pv2grid', 'bess2load', 'bess2grid', 'grid2load', 'grid2bess', 'pv2curtail']
subset_data = results_econ[columns_of_interest]

# Melt the DataFrame to long format
melted_data = subset_data.melt(id_vars=['PV_Load_M', 'BESS_Load_M', 'T_BESS'], var_name='source', value_name='value')

# Plot stacked bar chart
quant= alt.Chart(
    melted_data,
    title= alt.TitleParams(
        text= 'BESS-Load Multiples, binned',
        subtitle= 'Each bin represents a BESS-Load multiple, within which bars are offset for each PV-Load multiple according to the BESS duration capacity (left-to-right: 0 - 8 hours)',
        anchor= 'middle',
        fontSize= 30,
        subtitleFontSize= 20
    )
).mark_bar().encode(
    x=alt.X('PV_Load_M:O', title='PV-Load Multiples', axis=alt.Axis(offset= -5)),
    #x2=alt.X2('PV_Load_M:O', title='PV-Load Multiple'),# axis=alt.Axis(offset=-5)),
    #x=alt.X('T_BESS:N', title= 'BESS Duration', axis= alt.Axis(offset= -5)),
    xOffset= 'T_BESS:N',
    y=alt.Y('sum(value):Q', title='Quantity of Power flows', axis= alt.Axis(labels= False)).stack('normalize'),
    color=alt.Color('source:N', legend= alt.Legend(title= 'Power flows')),
    column= alt.Column('BESS_Load_M:N', title= '', header= alt.Header(labelFontSize= 20)),
    tooltip=['PV_Load_M', 'BESS_Load_M', 'T_BESS', 'source', 'value'],
    #size= alt.Size('T_BESS:Q').scale(domain= (-0.1, 2)),
    
).properties(
    width=300,
    height=600
).configure_axis(
    labelFontSize= 15,
    titleFontSize= 20,
).configure_legend(
    labelFontSize= 20,
    titleFontSize= 20
)
# num= alt.Chart(melted_data).mark_bar().encode(
#     x=alt.X('PV_Load_M:O', title='PV-Load Multiple', axis=alt.Axis(offset=-5)),
#     y=alt.Y('sum(value):Q', title='Quantity'),
#     color='source:N',
#     column= alt.Column('BESS_Load_M:N', title= 'BESS-Load Multiple, binned'),
#     tooltip=['PV_Load_M', 'BESS_Load_M', 'source', 'value'],
#     size= alt.Size('T_BESS:N').scale(domain= (-0.1, 2))
# ).properties(
#     width=300,
#     height=600
# )

quant

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [30]:
# Select only the columns of interest
columns_of_interest = ['PV_Load_M', 'BESS_Load_M', 'T_BESS', 'pv2load', 'pv2bess', 'pv2grid', 'bess2load', 'bess2grid', 'grid2load', 'grid2bess', 'pv2curtail']
subset_data = results_env[columns_of_interest]

# Melt the DataFrame to long format
melted_data = subset_data.melt(id_vars=['PV_Load_M', 'BESS_Load_M', 'T_BESS'], var_name='source', value_name='value')

# Plot stacked bar chart
quant= alt.Chart(
    melted_data,
    title= alt.TitleParams(
        text= 'BESS-Load Multiples, binned',
        subtitle= 'Each bin represents a BESS-Load multiple, within which bars are offset for each PV-Load multiple according to the BESS duration capacity (left-to-right: 0 - 8 hours)',
        anchor= 'middle',
        fontSize= 30,
        subtitleFontSize= 20
    )
).mark_bar().encode(
    x=alt.X('PV_Load_M:O', title='PV-Load Multiples', axis=alt.Axis(offset= -5)),
    #x2=alt.X2('PV_Load_M:O', title='PV-Load Multiple'),# axis=alt.Axis(offset=-5)),
    #x=alt.X('T_BESS:N', title= 'BESS Duration', axis= alt.Axis(offset= -5)),
    xOffset= 'T_BESS:N',
    y=alt.Y('sum(value):Q', title='Quantity of Power flows', axis= alt.Axis(labels= False)).stack('normalize'),
    color=alt.Color('source:N', legend= alt.Legend(title= 'Power flows')),
    column= alt.Column('BESS_Load_M:N', title= '', header= alt.Header(labelFontSize= 20)),
    tooltip=['PV_Load_M', 'BESS_Load_M', 'T_BESS', 'source', 'value'],
    #size= alt.Size('T_BESS:Q').scale(domain= (-0.1, 2)),
    
).properties(
    width=300,
    height=600
).configure_axis(
    labelFontSize= 15,
    titleFontSize= 20,
).configure_legend(
    labelFontSize= 20,
    titleFontSize= 20
)
# num= alt.Chart(melted_data).mark_bar().encode(
#     x=alt.X('PV_Load_M:O', title='PV-Load Multiple', axis=alt.Axis(offset=-5)),
#     y=alt.Y('sum(value):Q', title='Quantity'),
#     color='source:N',
#     column= alt.Column('BESS_Load_M:N', title= 'BESS-Load Multiple, binned'),
#     tooltip=['PV_Load_M', 'BESS_Load_M', 'source', 'value'],
#     size= alt.Size('T_BESS:N').scale(domain= (-0.1, 2))
# ).properties(
#     width=300,
#     height=600
# )

quant

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

### Quick-Extraction Pareto Front

In [31]:
# Assuming you want to minimize both objectives
pareto_points_lcoe_revenue_1 = paretoset.paretoset(results[['lcoe_revenue', 'ress']].values, sense=["min", "min"])
pareto_points_lcoe_revenue_2 = paretoset.paretoset(results[['lcoe_revenue', 'ress']].values, sense=["min", "max"])
pareto_points_lco2_abatement_1 = paretoset.paretoset(results[['lco2_abatement', 'ress']].values, sense=["min", "min"])
pareto_points_lco2_abatement_2 = paretoset.paretoset(results[['lco2_abatement', 'ress']].values, sense=["min", "max"])

pareto_points_lcoe_investor_1 = paretoset.paretoset(results[['lcoe_investor', 'ress']].values, sense=["min", "min"])
pareto_points_lcoe_investor_2 = paretoset.paretoset(results[['lcoe_investor', 'ress']].values, sense=["min", "max"])
pareto_points_lco2_investor_1 = paretoset.paretoset(results[['lco2_investor', 'ress']].values, sense=["min", "min"])
pareto_points_lco2_investor_2 = paretoset.paretoset(results[['lco2_investor', 'ress']].values, sense=["min", "max"])

pareto_points_lcoe_enduser_1 = paretoset.paretoset(results[['lcoe_enduser', 'ress']].values, sense=["min", "min"])
pareto_points_lcoe_enduser_2 = paretoset.paretoset(results[['lcoe_enduser', 'ress']].values, sense=["min", "max"])
pareto_points_lco2_enduser_1 = paretoset.paretoset(results[['lco2_enduser', 'ress']].values, sense=["min", "min"])
pareto_points_lco2_enduser_2 = paretoset.paretoset(results[['lco2_enduser', 'ress']].values, sense=["min", "max"])

# Extract Pareto-optimal points
pareto_front_lcoe_revenue = pd.merge(results.iloc[pareto_points_lcoe_revenue_1], results.iloc[pareto_points_lcoe_revenue_2], how='outer')
pareto_front_lco2_abatement = pd.merge(results.iloc[pareto_points_lco2_abatement_1], results.iloc[pareto_points_lco2_abatement_2], how='outer')

pareto_front_lcoe_investor = pd.merge(results.iloc[pareto_points_lcoe_investor_1], results.iloc[pareto_points_lcoe_investor_2], how='outer')
pareto_front_lco2_investor = pd.merge(results.iloc[pareto_points_lco2_investor_1], results.iloc[pareto_points_lco2_investor_2], how='outer')

pareto_front_lcoe_enduser = pd.merge(results.iloc[pareto_points_lcoe_enduser_1], results.iloc[pareto_points_lcoe_enduser_2], how='outer')
pareto_front_lco2_enduser = pd.merge(results.iloc[pareto_points_lco2_enduser_1], results.iloc[pareto_points_lco2_enduser_2], how='outer')

In [32]:
pareto_lcoe_revenue= alt.Chart(pareto_front_lcoe_revenue).mark_line(
    color= 'black',
    size= 1.10
).encode(
    x= alt.X('ress:Q'),
    y= alt.Y('lcoe_revenue:Q')
)
pareto_lco2_abatement= alt.Chart(pareto_front_lco2_abatement).mark_line(
    color= 'black',
    size= 1.10
).encode(
    x= alt.X('ress:Q'),
    y= alt.Y('lco2_abatement:Q')
)

pareto_lcoe_investor= alt.Chart(pareto_front_lcoe_investor).mark_line(
    color= 'black',
    size= 1.10
).encode(
    x= alt.X('ress:Q'),
    y= alt.Y('lcoe_investor:Q')
)
pareto_lco2_investor= alt.Chart(pareto_front_lco2_investor).mark_line(
    color= 'black',
    size= 1.10
).encode(
    x= alt.X('ress:Q'),
    y= alt.Y('lco2_investor:Q')
)

pareto_lcoe_enduser= alt.Chart(pareto_front_lcoe_enduser).mark_line(
    color= 'black',
    size= 1.10
).encode(
    x= alt.X('ress:Q'),
    y= alt.Y('lcoe_enduser:Q')
)
pareto_lco2_enduser= alt.Chart(pareto_front_lco2_enduser).mark_line(
    color= 'black',
    size= 1.10
).encode(
    x= alt.X('ress:Q'),
    y= alt.Y('lco2_enduser:Q')
)

In [33]:
alt.hconcat(
    alt.layer(
        points_lcoe_revenue,
        lines_lcoe_revenue_econ,
        lines_lcoe_revenue_env,
        pareto_lcoe_revenue
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    ),
    alt.layer(
        points_lco2_abatement,
        lines_lco2_abatement_econ,
        lines_lco2_abatement_env,
        pareto_lco2_abatement
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    )
).resolve_scale(
    #legend= 'independent',
    #scale= 'independent',
    shape= 'independent',
    color= 'independent'
)

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/ana

alt.HConcatChart(...)

In [34]:
alt.hconcat(
    alt.layer(
        points_lcoe_investor,
        lines_lcoe_investor_econ,
        lines_lcoe_investor_env,
        pareto_lcoe_investor
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    ),
    alt.layer(
        points_lco2_investor,
        lines_lco2_investor_econ,
        lines_lco2_investor_env,
        pareto_lco2_investor
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    )
).resolve_scale(
    #legend= 'independent',
    #scale= 'independent',
    shape= 'independent',
    color= 'independent'
)

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/ana

alt.HConcatChart(...)

In [35]:
alt.hconcat(
    alt.layer(
        points_lcoe_enduser,
        lines_lcoe_enduser_econ,
        lines_lcoe_enduser_env,
        chart_band_price,
        chart_rule_price_quartiles,
        chart_rule_price_avg,
        pareto_lcoe_enduser
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    ),
    alt.layer(
        points_lco2_enduser,
        lines_lco2_enduser_econ,
        lines_lco2_enduser_env,
        chart_band_co2,
        chart_rule_co2_quartiles,
        chart_rule_co2_avg,
        pareto_lco2_enduser
    ).resolve_scale(
        color= 'independent',
        size= 'independent'
    )
).resolve_scale(
    #legend= 'independent',
    #scale= 'independent',
    shape= 'independent',
    color= 'independent'
)

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/ana

alt.HConcatChart(...)

In [36]:
## Plot only the points on the pareto fronts to see what multiples for PV, BESS, and BESS duration are on the front
left_up_revenue= alt.Chart(pareto_front_lcoe_revenue).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lcoe_revenue:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values())
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'BESS_Load_M:Q',
        legend= alt.Legend(
            #title='Savings',
            #title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0,
            # titleOpacity=0
        )
    ).scale(
        scheme='redblue',
        #domain= (-1500000, 1500000)
    ),
    size= alt.Size(
        'E_BESS:Q',
        legend= alt.Legend(
            #title= '',
        )
    )
)
right_up_abatement= alt.Chart(pareto_front_lco2_abatement).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lco2_abatement:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values()),
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'BESS_Load_M:Q',
        legend= alt.Legend(
            #title='CO2eq Avoided',
            # title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (-800000, 500000)
    ),
    size= alt.Size(
        'E_BESS:Q'
    )
)


left_down_revenue= alt.Chart(pareto_front_lcoe_revenue).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lcoe_revenue:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values())
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            #title='Savings',
            #title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0,
            # titleOpacity=0
        )
    ).scale(
        scheme='redblue',
        #domain= (-1500000, 1500000)
    ),
    size= alt.Size(
        'E_BESS:Q',
        legend= alt.Legend(
            #title= '',
        )
    )
)
right_down_abatement= alt.Chart(pareto_front_lco2_abatement).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lco2_abatement:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values()),
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            #title='CO2eq Avoided',
            # title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (-800000, 500000)
    ),
    size= alt.Size(
        'E_BESS:Q'
    )
)


alt.vconcat(
    alt.hconcat(
        left_up_revenue,
        right_up_abatement
    ).resolve_scale(
        color= 'independent',
        shape= 'independent',
        size= 'independent'
    ),
    alt.hconcat(
        left_down_revenue,
        right_down_abatement
    ).resolve_scale(
        color= 'independent',
        shape= 'independent',
        size= 'independent'
    )
)

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/ana

alt.VConcatChart(...)

In [37]:
## Plot only the points on the pareto fronts to see what multiples for PV, BESS, and BESS duration are on the front
left_up_investor= alt.Chart(pareto_front_lcoe_investor).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lcoe_investor:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values())
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'BESS_Load_M:Q',
        legend= alt.Legend(
            #title='Savings',
            #title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0,
            # titleOpacity=0
        )
    ).scale(
        scheme='redblue',
        #domain= (-1500000, 1500000)
    ),
    size= alt.Size(
        'E_BESS:Q',
        legend= alt.Legend(
            #title= '',
        )
    )
)
right_up_investor= alt.Chart(pareto_front_lco2_investor).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lco2_investor:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values()),
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'BESS_Load_M:Q',
        legend= alt.Legend(
            #title='CO2eq Avoided',
            # title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (-800000, 500000)
    ),
    size= alt.Size(
        'E_BESS:Q'
    )
)


left_down_investor= alt.Chart(pareto_front_lcoe_investor).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lcoe_investor:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values())
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            #title='Savings',
            #title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0,
            # titleOpacity=0
        )
    ).scale(
        scheme='redblue',
        #domain= (-1500000, 1500000)
    ),
    size= alt.Size(
        'E_BESS:Q',
        legend= alt.Legend(
            #title= '',
        )
    )
)
right_down_investor= alt.Chart(pareto_front_lco2_investor).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lco2_investor:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values()),
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            #title='CO2eq Avoided',
            # title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (-800000, 500000)
    ),
    size= alt.Size(
        'E_BESS:Q'
    )
)


alt.vconcat(
    alt.hconcat(
        left_up_investor,
        right_up_investor
    ).resolve_scale(
        color= 'independent',
        shape= 'independent',
        size= 'independent'
    ),
    alt.hconcat(
        left_down_investor,
        right_down_investor
    ).resolve_scale(
        color= 'independent',
        shape= 'independent',
        size= 'independent'
    )
)

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/ana

alt.VConcatChart(...)

In [38]:
## Plot only the points on the pareto fronts to see what multiples for PV, BESS, and BESS duration are on the front
left_up_enduser= alt.Chart(pareto_front_lcoe_enduser).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lcoe_enduser:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values())
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'BESS_Load_M:Q',
        legend= alt.Legend(
            #title='Savings',
            #title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0,
            # titleOpacity=0
        )
    ).scale(
        scheme='redblue',
        #domain= (-1500000, 1500000)
    ),
    size= alt.Size(
        'savings:Q',
        legend= alt.Legend(
            #title= '',
        )
    )
)
right_up_enduser= alt.Chart(pareto_front_lco2_enduser).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lco2_enduser:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values()),
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'BESS_Load_M:Q',
        legend= alt.Legend(
            #title='CO2eq Avoided',
            # title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (-800000, 500000)
    ),
    size= alt.Size(
        'co2avoided:Q'
    )
)


left_down_enduser= alt.Chart(pareto_front_lcoe_enduser).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lcoe_enduser:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values())
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            #title='Savings',
            #title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0,
            # titleOpacity=0
        )
    ).scale(
        scheme='redblue',
        #domain= (-1500000, 1500000)
    ),
    size= alt.Size(
        'savings:Q',
        legend= alt.Legend(
            #title= '',
        )
    )
)
right_down_enduser= alt.Chart(pareto_front_lco2_enduser).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lco2_enduser:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values()),
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            #title='CO2eq Avoided',
            # title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0
        )
    ).scale(
        scheme='redyellowgreen',
        #domain= (-800000, 500000)
    ),
    size= alt.Size(
        'co2avoided:Q'
    )
)


alt.vconcat(
    alt.hconcat(
        (left_up_enduser + chart_band_price + chart_rule_price_quartiles + chart_rule_price_avg),
        (right_up_enduser + chart_band_co2 + chart_rule_co2_quartiles + chart_rule_co2_avg)
    ).resolve_scale(
        color= 'independent',
        shape= 'independent',
        size= 'independent'
    ),
    alt.hconcat(
        (left_down_enduser + chart_band_price + chart_rule_price_quartiles + chart_rule_price_avg),
        (right_down_enduser + chart_band_co2 + chart_rule_co2_quartiles + chart_rule_co2_avg)
    ).resolve_scale(
        color= 'independent',
        shape= 'independent',
        size= 'independent'
    )
)

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/ana

alt.VConcatChart(...)

In [39]:
## Plot only the points on the pareto fronts to see what multiples for PV, BESS, and BESS duration are on the front
left_up_investor= alt.Chart(pareto_front_lcoe_investor).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lcoe_investor:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values())
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'BESS_Load_M:Q',
        legend= alt.Legend(
            #title='Savings',
            #title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0,
            # titleOpacity=0
        )
    ).scale(
        scheme='plasma',
        #domain= (-1500000, 1500000)
    ),
    size= alt.Size(
        'E_BESS:Q',
        legend= alt.Legend(
            #title= '',
        )
    )
)
right_up_investor= alt.Chart(pareto_front_lco2_investor).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lco2_investor:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values()),
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'BESS_Load_M:Q',
        legend= alt.Legend(
            #title='CO2eq Avoided',
            # title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0
        )
    ).scale(
        scheme='plasma',
        #domain= (-800000, 500000)
    ),
    size= alt.Size(
        'E_BESS:Q'
    )
)


left_down_investor= alt.Chart(pareto_front_lcoe_investor).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lcoe_investor:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values())
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            #title='Savings',
            #title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0,
            # titleOpacity=0
        )
    ).scale(
        scheme='turbo',
        #domain= (-1500000, 1500000)
    ),
    size= alt.Size(
        'E_BESS:Q',
        legend= alt.Legend(
            #title= '',
        )
    )
)
right_down_investor= alt.Chart(pareto_front_lco2_investor).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lco2_investor:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values()),
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            #title='CO2eq Avoided',
            # title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0
        )
    ).scale(
        scheme='turbo',
        #domain= (-800000, 500000)
    ),
    size= alt.Size(
        'E_BESS:Q'
    )
)


alt.vconcat(
    alt.hconcat(
        left_up_investor,
        right_up_investor
    ).resolve_scale(
        color= 'independent',
        shape= 'independent',
        size= 'independent'
    ),
    alt.hconcat(
        left_down_investor,
        right_down_investor
    ).resolve_scale(
        color= 'independent',
        shape= 'independent',
        size= 'independent'
    )
)

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/ana

alt.VConcatChart(...)

In [40]:
## Plot only the points on the pareto fronts to see what multiples for PV, BESS, and BESS duration are on the front
left_up_enduser= alt.Chart(pareto_front_lcoe_enduser).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lcoe_enduser:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values())
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'BESS_Load_M:Q',
        legend= alt.Legend(
            #title='Savings',
            #title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0,
            # titleOpacity=0
        )
    ).scale(
        scheme='plasma',
        #domain= (-1500000, 1500000)
    ),
    size= alt.Size(
        'savings:Q',
        legend= alt.Legend(
            #title= '',
        )
    )
)
right_up_enduser= alt.Chart(pareto_front_lco2_enduser).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lco2_enduser:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values()),
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'BESS_Load_M:Q',
        legend= alt.Legend(
            #title='CO2eq Avoided',
            # title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0
        )
    ).scale(
        scheme='plasma',
        #domain= (-800000, 500000)
    ),
    size= alt.Size(
        'co2avoided:Q'
    )
)


left_down_enduser= alt.Chart(pareto_front_lcoe_enduser).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lcoe_enduser:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values())
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            #title='Savings',
            #title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0,
            # titleOpacity=0
        )
    ).scale(
        scheme='turbo',
        #domain= (-1500000, 1500000)
    ),
    size= alt.Size(
        'savings:Q',
        legend= alt.Legend(
            #title= '',
        )
    )
)
right_down_enduser= alt.Chart(pareto_front_lco2_enduser).mark_point(
    size= 150,
    strokeWidth= 1
).encode(
    alt.X(
        'ress:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    alt.Y(
        'lco2_enduser:Q',
        # axis= alt.Axis(
        #     title='',
        #     labelOpacity= 0
        # )
    ),
    shape= alt.Shape(
        'label:N',
        scale=alt.Scale(
            domain=list(shape_values.keys()),
            range=list(shape_values.values()),
        ),
        legend= alt.Legend(
            title= 'Dispatch Optimisation'
        )
    ),
    color= alt.Color(
        'PV_Load_M:Q',
        legend= alt.Legend(
            #title='CO2eq Avoided',
            # title='',
            # labelOpacity= 0,
            # symbolOpacity= 0,
            # gradientOpacity= 0
        )
    ).scale(
        scheme='turbo',
        #domain= (-800000, 500000)
    ),
    size= alt.Size(
        'co2avoided:Q'
    )
)


alt.vconcat(
    alt.hconcat(
        (left_up_enduser + chart_band_price + chart_rule_price_quartiles + chart_rule_price_avg),
        (right_up_enduser + chart_band_co2 + chart_rule_co2_quartiles + chart_rule_co2_avg)
    ).resolve_scale(
        color= 'independent',
        shape= 'independent',
        size= 'independent'
    ),
    alt.hconcat(
        (left_down_enduser + chart_band_price + chart_rule_price_quartiles + chart_rule_price_avg),
        (right_down_enduser + chart_band_co2 + chart_rule_co2_quartiles + chart_rule_co2_avg)
    ).resolve_scale(
        color= 'independent',
        shape= 'independent',
        size= 'independent'
    )
)

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/ana

alt.VConcatChart(...)

In [41]:
pareto_front_lcoe_enduser['ress']

0    0.0
Name: ress, dtype: float64

### Specifying separate Paretto fronts for economic and environmental dispatch

In [44]:
results_econ= pd.read_csv( '../data/processed/CaseB/results_econ.csv',  sep= ';', )
results_env= pd.read_csv(
    '../data/processed/CaseB/results_env.csv',
    sep= ';',
)
results= pd.read_csv(
    '../data/processed/CaseB/results.csv',
    sep= ';',
)

In [45]:
# Assuming you want to minimize both objectives or mix
# One front is for min-min and one for min-max, so we end up with left and right part of the Paretto front (this is datapoint dependant)
pareto_points_lcoe_econ_revenue_1 = paretoset.paretoset(results_econ[['lcoe_revenue', 'ress']].values, sense=["min", "min"])
pareto_points_lcoe_econ_revenue_2 = paretoset.paretoset(results_econ[['lcoe_revenue', 'ress']].values, sense=["min", "max"])
pareto_points_lco2_econ_abatement_1 = paretoset.paretoset(results_econ[['lco2_abatement', 'ress']].values, sense=["min", "min"])
pareto_points_lco2_econ_abatement_2 = paretoset.paretoset(results_econ[['lco2_abatement', 'ress']].values, sense=["min", "max"])

pareto_points_lcoe_econ_investor_1 = paretoset.paretoset(results_econ[['lcoe_investor', 'ress']].values, sense=["min", "min"])
pareto_points_lcoe_econ_investor_2 = paretoset.paretoset(results_econ[['lcoe_investor', 'ress']].values, sense=["min", "max"])
pareto_points_lco2_econ_investor_1 = paretoset.paretoset(results_econ[['lco2_investor', 'ress']].values, sense=["min", "min"])
pareto_points_lco2_econ_investor_2 = paretoset.paretoset(results_econ[['lco2_investor', 'ress']].values, sense=["min", "max"])

pareto_points_lcoe_econ_enduser_1 = paretoset.paretoset(results_econ[['lcoe_enduser', 'ress']].values, sense=["min", "min"])
pareto_points_lcoe_econ_enduser_2 = paretoset.paretoset(results_econ[['lcoe_enduser', 'ress']].values, sense=["min", "max"])
pareto_points_lco2_econ_enduser_1 = paretoset.paretoset(results_econ[['lco2_enduser', 'ress']].values, sense=["min", "min"])
pareto_points_lco2_econ_enduser_2 = paretoset.paretoset(results_econ[['lco2_enduser', 'ress']].values, sense=["min", "max"])

# Extract Pareto-optimal points
pareto_front_lcoe_econ_revenue = pd.merge(results_econ.iloc[pareto_points_lcoe_econ_revenue_1], results_econ.iloc[pareto_points_lcoe_econ_revenue_2], how='outer')
pareto_front_lco2_econ_abatement = pd.merge(results_econ.iloc[pareto_points_lco2_econ_abatement_1], results_econ.iloc[pareto_points_lco2_econ_abatement_2], how='outer')

pareto_front_lcoe_econ_investor = pd.merge(results_econ.iloc[pareto_points_lcoe_econ_investor_1], results_econ.iloc[pareto_points_lcoe_econ_investor_2], how='outer')
pareto_front_lco2_econ_investor = pd.merge(results_econ.iloc[pareto_points_lco2_econ_investor_1], results_econ.iloc[pareto_points_lco2_econ_investor_2], how='outer')

pareto_front_lcoe_econ_enduser = pd.merge(results_econ.iloc[pareto_points_lcoe_econ_enduser_1], results_econ.iloc[pareto_points_lcoe_econ_enduser_2], how='outer')
pareto_front_lco2_econ_enduser = pd.merge(results_econ.iloc[pareto_points_lco2_econ_enduser_1], results_econ.iloc[pareto_points_lco2_econ_enduser_2], how='outer')



# Assuming you want to minimize both objectives
pareto_points_lcoe_env_revenue_1 = paretoset.paretoset(results_env[['lcoe_revenue', 'ress']].values, sense=["min", "min"])
pareto_points_lcoe_env_revenue_2 = paretoset.paretoset(results_env[['lcoe_revenue', 'ress']].values, sense=["min", "max"])
pareto_points_lco2_env_abatement_1 = paretoset.paretoset(results_env[['lco2_abatement', 'ress']].values, sense=["min", "min"])
pareto_points_lco2_env_abatement_2 = paretoset.paretoset(results_env[['lco2_abatement', 'ress']].values, sense=["min", "max"])

pareto_points_lcoe_env_investor_1 = paretoset.paretoset(results_env[['lcoe_investor', 'ress']].values, sense=["min", "min"])
pareto_points_lcoe_env_investor_2 = paretoset.paretoset(results_env[['lcoe_investor', 'ress']].values, sense=["min", "max"])
pareto_points_lco2_env_investor_1 = paretoset.paretoset(results_env[['lco2_investor', 'ress']].values, sense=["min", "min"])
pareto_points_lco2_env_investor_2 = paretoset.paretoset(results_env[['lco2_investor', 'ress']].values, sense=["min", "max"])

pareto_points_lcoe_env_enduser_1 = paretoset.paretoset(results_env[['lcoe_enduser', 'ress']].values, sense=["min", "min"])
pareto_points_lcoe_env_enduser_2 = paretoset.paretoset(results_env[['lcoe_enduser', 'ress']].values, sense=["min", "max"])
pareto_points_lco2_env_enduser_1 = paretoset.paretoset(results_env[['lco2_enduser', 'ress']].values, sense=["min", "min"])
pareto_points_lco2_env_enduser_2 = paretoset.paretoset(results_env[['lco2_enduser', 'ress']].values, sense=["min", "max"])

# Extract Pareto-optimal points
pareto_front_lcoe_env_revenue = pd.merge(results_env.iloc[pareto_points_lcoe_env_revenue_1], results_env.iloc[pareto_points_lcoe_env_revenue_2], how='outer')
pareto_front_lco2_env_abatement = pd.merge(results_env.iloc[pareto_points_lco2_env_abatement_1], results_env.iloc[pareto_points_lco2_env_abatement_2], how='outer')

pareto_front_lcoe_env_investor = pd.merge(results_env.iloc[pareto_points_lcoe_env_investor_1], results_env.iloc[pareto_points_lcoe_env_investor_2], how='outer')
pareto_front_lco2_env_investor = pd.merge(results_env.iloc[pareto_points_lco2_env_investor_1], results_env.iloc[pareto_points_lco2_env_investor_2], how='outer')

pareto_front_lcoe_env_enduser = pd.merge(results_env.iloc[pareto_points_lcoe_env_enduser_1], results_env.iloc[pareto_points_lcoe_env_enduser_2], how='outer')
pareto_front_lco2_env_enduser = pd.merge(results_env.iloc[pareto_points_lco2_env_enduser_1], results_env.iloc[pareto_points_lco2_env_enduser_2], how='outer')

In [46]:
lcoe_econ_investor= alt.Chart(pareto_front_lcoe_econ_investor).mark_line(
    strokeDash= (0, 0),
    color= 'red',
    opacity= 0.75
).encode(
    x= 'ress:Q',
    y= 'lcoe_investor:Q'
)
lcoe_env_investor= alt.Chart(pareto_front_lcoe_env_investor).mark_line(
    strokeDash= (0, 0),
    color= 'green',
    opacity= 0.75
).encode(
    x= 'ress:Q',
    y= 'lcoe_investor:Q'
)
lcoe_econ_enduser= alt.Chart(pareto_front_lcoe_econ_enduser).mark_line(
    strokeDash= (5, 5),
    color= 'red',
    opacity= 0.75
).encode(
    x= 'ress:Q',
    y= 'lcoe_enduser:Q'
)
lcoe_env_enduser= alt.Chart(pareto_front_lcoe_env_enduser).mark_line(
    strokeDash= (5, 5),
    color= 'green',
    opacity= 0.75
).encode(
    x= 'ress:Q',
    y= 'lcoe_enduser:Q'
)


lco2_econ_investor= alt.Chart(pareto_front_lco2_econ_investor).mark_line(
    strokeDash= (0, 0),
    color= 'red',
    opacity= 0.75
).encode(
    x= 'ress:Q',
    y= 'lco2_investor:Q'
)
lco2_env_investor= alt.Chart(pareto_front_lco2_env_investor).mark_line(
    strokeDash= (0, 0),
    color= 'green',
    opacity= 0.75
).encode(
    x= 'ress:Q',
    y= 'lco2_investor:Q'
)
lco2_econ_enduser= alt.Chart(pareto_front_lco2_econ_enduser).mark_line(
    strokeDash= (5, 5),
    color= 'red',
    opacity= 0.75
).encode(
    x= 'ress:Q',
    y= 'lco2_enduser:Q'
)
lco2_env_enduser= alt.Chart(pareto_front_lco2_env_enduser).mark_line(
    strokeDash= (5, 5),
    color= 'green',
    opacity= 0.75
).encode(
    x= 'ress:Q',
    y= 'lco2_enduser:Q'
)


alt.hconcat(
    alt.layer(
        lcoe_econ_investor,
        lcoe_econ_enduser,
        lcoe_env_investor,
        lcoe_env_enduser
    ),
    alt.layer(
        lco2_econ_investor,
        lco2_econ_enduser,
        lco2_env_investor,
        lco2_env_enduser
    )
)

/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/anaconda3/envs/BESS/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/leo/ana

alt.HConcatChart(...)